In [10]:
import numpy as np
  
def find_nonzero_row(matrix, pivot_row, col):
    nrows = matrix.shape[0]
    for row in range(pivot_row, nrows):
        if matrix[row, col] != 0:
            return row
    return None
 
def swap_rows(matrix, row1, row2):
    matrix[[row1, row2]] = matrix[[row2, row1]]
 
def make_pivot_one(matrix, pivot_row, col):
    pivot_element = matrix[pivot_row, col]
    matrix[pivot_row] /= pivot_element
 
def eliminate_below(matrix, pivot_row, col):
    nrows = matrix.shape[0]
    for row in range(pivot_row + 1, nrows):
        if (matrix[row][col] != 0):
            factor = matrix[row, col]
            matrix[row] -= factor * matrix[pivot_row]

def eliminate_above(matrix, pivot_row, col):
    for row in range(0, pivot_row):
        if (matrix[row][col] != 0):
            factor = matrix[row, col]
            matrix[row] -= factor * matrix[pivot_row]
 
def row_echelon_form(matrix):
    ncols = matrix.shape[1]
    pivot_row = 0
    
    for col in range(ncols):
        nonzero_row = find_nonzero_row(matrix, pivot_row, col)
        if nonzero_row is not None:
            if (pivot_row != nonzero_row):
                swap_rows(matrix, pivot_row, nonzero_row)
            make_pivot_one(matrix, pivot_row, col)
            eliminate_below(matrix, pivot_row, col)
            pivot_row += 1
    return matrix

def reduced_row_echelon_form(refMatrix):
    nrows = refMatrix.shape[0]
    ncols = refMatrix.shape[1]
    pivot_row = 1
    for col in range(1,ncols):
        if (refMatrix[pivot_row][col] == 1):
            eliminate_above(refMatrix, pivot_row, col)
        else:
            continue
        pivot_row += 1
        if(pivot_row >= nrows):
            break
    return refMatrix

def minus_one_padding(rrefMatrix):
    nRows = rrefMatrix.shape[0]
    nCols = rrefMatrix.shape[1]
    
    # Find pivot and non pivot columns
    pivots = pivot_columns(rrefMatrix)
    nonPivots = np.array(list(set(np.arange(rrefMatrix.shape[1] -1)) - set(pivots)))

    for i in nonPivots:
        rowToInsert = np.zeros(nCols)
        rowToInsert[i] = -1
        rrefMatrix = np.insert(rrefMatrix, i, rowToInsert, axis=0)
    
    res = rrefMatrix[~np.all(rrefMatrix == 0, axis=1)]
    return res   

def pivot_columns(rrefMatrix):
    nrows = rrefMatrix.shape[0]
    ncols = rrefMatrix.shape[1]
    pivot_row = 0
    pivot_columns = []
    for col in range(ncols):
        if (rrefMatrix[pivot_row][col] == 1):
            pivot_columns.append(col)
        else:
            continue
        pivot_row += 1
        if(pivot_row >= nrows):
            break
    return np.array(pivot_columns)
  
def is_row_echelon_form(matrix):
    if not matrix.any():
        return False
 
    rows = matrix.shape[0]
    cols = matrix.shape[1]
    prev_leading_col = -1
 
    for row in range(rows):
        leading_col_found = False
        for col in range(cols):
            if matrix[row, col] != 0:
                if col <= prev_leading_col:
                    return False
                prev_leading_col = col
                leading_col_found = True
                break
        if not leading_col_found and any(matrix[row, col] != 0 for col in range(cols)):
            return False
    return True

def SolveSystem(A,B):
    # Augmented Matrix A|B
    A_augmented = np.hstack((A,B))

    nRows = A.shape[0]
    nCols = A.shape[1]

    # 2. REF   
    refA = row_echelon_form(A)
    refAugmentedAB = row_echelon_form(A_augmented)

    if not is_row_echelon_form(refA) or not is_row_echelon_form(refAugmentedAB):
        print("Not in REF!")
        return None
    
    pivotsA = pivot_columns(refA)
    pivotsAB = pivot_columns(refAugmentedAB)

    if (pivotsA.size < pivotsAB.size):
        print("No solution. The system is inconsistent.")
        return None
    
    print("RREF matrix for A|B")
    rref = reduced_row_echelon_form(refAugmentedAB)
    print(rref)
    
    if (nRows < nCols or pivotsA.size < nCols):
        print("The system has many solutions")    
        minusOnePaddedMatrix = minus_one_padding(refAugmentedAB)
        particularSolution = minusOnePaddedMatrix[:,nCols]
        nonPivots = np.array(list(set(np.arange(refAugmentedAB.shape[1] -1)) - set(pivotsA)))

        generalSolutions = np.zeros((nonPivots.size, nCols))
        for i in range(nonPivots.size):
            generalSolutions[i] = minusOnePaddedMatrix[0:,nonPivots[i]]
        print("The particular solution")
        print(particularSolution)
        print("The general solutions to AX = 0")
        print(generalSolutions)
    
    if (pivotsA.size == nCols):
        print("The system has unique solution. The last column in RREF A|B is the solution")
        print(rref[:,nCols])
 

# 1. INPUT

A = np.array([[-2,4,-2,-1,4], [4,-8,3,-3,1], [1,-2,1,-1,1], [1,-2,0,-3,4]], dtype=np.float64)
B = np.array([[-3],[2],[0],[-1]], dtype=np.float64)

SolveSystem(A,B)

RREF matrix for A|B
[[ 1. -2.  0.  0. -2.  2.]
 [ 0.  0.  1.  0.  1. -1.]
 [-0. -0. -0.  1. -2.  1.]
 [ 0.  0.  0.  0.  0.  0.]]
The system has many solutions
The particular solution
[ 2.  0. -1.  1.  0.]
The general solutions to AX = 0
[[-2. -1.  0. -0.  0.]
 [-2.  0.  1. -2. -1.]]
